In [2]:
import telebot
from telebot import types
import pickle
import pandas as pd
import numpy as np

In [13]:
with open('token.txt') as f:
    token = f.readline()
bot = telebot.TeleBot(token)

with open('model_price.pkl', 'rb') as f:
    model_price = pickle.load(f)
with open('model_links.pkl', 'rb') as f:
    model_links = pickle.load(f)

df = pd.read_csv('df.csv', index_col=0)

In [52]:
user_dict = {}


class Estate:
    def __init__(self, num):
        self.rooms = num
        self.square = None
        self.floor = None
        self.max_floor = None
        self.year = None
        
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Предоставить данные о квартире")
    markup.add(btn1)
    msg = bot.reply_to(message, """\
Здравствуйте! Я - бот, который подскажет цену на вашу квартиру в г. Пермь и отправит ссылки на похожие варинты квартир.
Мне будут необходимы следующие данные:
- количество комнат
- площадь квартиры
- этаж
- этажность дома
- год постройки дома.
""", reply_markup=markup)
    bot.register_next_step_handler(msg, first_step)


def first_step(message):
    msg = bot.reply_to(message, 'Сколько комнат в квартире?')
    bot.register_next_step_handler(msg, process_rooms_step)


def process_rooms_step(message):
    try:
        chat_id = message.chat.id
        rooms = message.text
        if not rooms.isdigit():
            msg = bot.reply_to(message, 'Количество комнат должно быть числом.\nСколько комнат в квартире?')
            bot.register_next_step_handler(msg, process_rooms_step)
            return
        estate = Estate(rooms)
        user_dict[chat_id] = estate
        msg = bot.reply_to(message, 'Какая площадь квартиры?')
        bot.register_next_step_handler(msg, process_square_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_square_step(message):
    try:
        chat_id = message.chat.id
        square = message.text
        if not square.isdigit():
            msg = bot.reply_to(message, 'Площадь должна быть числом. Какая площадь квартиры?')
            bot.register_next_step_handler(msg, process_square_step)
            return
        estate = user_dict[chat_id]
        estate.square = square
        msg = bot.reply_to(message, 'Какой этаж?')
        bot.register_next_step_handler(msg, process_max_floor_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_max_floor_step(message):
    try:
        chat_id = message.chat.id
        floor = message.text
        if not floor.isdigit():
            msg = bot.reply_to(message, 'Этаж должен быть числом. Какой этаж?')
            bot.register_next_step_handler(msg, process_max_floor_step)
            return
        estate = user_dict[chat_id]
        estate.floor = floor
        msg = bot.reply_to(message, 'Сколько этажей дом?')
        bot.register_next_step_handler(msg, process_year_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')
        
def process_year_step(message):
    try:
        chat_id = message.chat.id
        max_floor = message.text
        if not max_floor.isdigit():
            msg = bot.reply_to(message, 'Этажность должна быть числом. Сколько этажей дом?')
            bot.register_next_step_handler(msg, process_balcony_step)
            return
        estate = user_dict[chat_id]
        estate.max_floor = max_floor
        msg = bot.reply_to(message, 'Какого года дом?')
        bot.register_next_step_handler(msg, process_ending_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_ending_step(message):
    try:
        chat_id = message.chat.id
        year = message.text
        if not year.isdigit():
            msg = bot.reply_to(message, 'Год должен быть числом. Какой год?')
            bot.register_next_step_handler(msg, process_ending_step)
            return
        estate = user_dict[chat_id]
        estate.year = year
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Узнать цену')
        msg = bot.reply_to(message, f'''Ваши данные по квартире:
- количество комнат: {estate.rooms}
- площадь квартиры: {estate.square}
- этаж: {estate.floor}
- этажность дома: {estate.max_floor}
- год постройки дома: {estate.year}''', reply_markup=markup)
        bot.register_next_step_handler(msg, predict_price)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def predict_price(message):
    try:
        chat_id = message.chat.id
        estate = user_dict[chat_id]
        
        X_pred = pd.DataFrame({'Rooms': [int(estate.rooms)], 
                       'Area': [float(estate.square)], 
                       'Floor': [int(estate.floor)], 
                       'Max_house_floor': [int(estate.max_floor)],
                       'Year_built': [int(estate.year)]
                       })
        
        price = int(model_price.predict(X_pred)[0])
        price_if = price
        price -= price % -10000
        price = str(price)
        if len(price) == 7:
            price = price[0] + ' ' + price[1:4] + ' ' + price[4:]
        elif len(price) == 8:
            price = price[:2] + ' ' + price[1:4] + ' ' + price[4:]
        
        neighbors = model_links.best_estimator_[1].kneighbors(X_pred, return_distance=False)[0]
        links = []
        for i in neighbors:
            if ((df.iloc[i, -7] - 900000) < price_if < (df.iloc[i, -7] + 900000)) and (df.iloc[i, -6] == X_pred.iloc[0, 0]):
                links.append(df.iloc[i, -1])
        bot.send_message(chat_id, f'''Ориентировочная стоимость вашей квартиры: <b>{price}</b> руб.
Похожие объявления:
{' '.join(links)}''', parse_mode='html')
    except Exception as e:
        bot.reply_to(message, 'Упс!')

In [53]:
bot.polling(none_stop=True)